If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
! pip install datasets transformers seqeval sentencepiece accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 

In [2]:
import transformers

print(transformers.__version__)

4.29.2


# Fine-tuning a model on a token classification task

In [3]:
task = "ner"
model_checkpoint = "sultan/BioM-BERT-PubMed-PMC-Large"
batch_size = 16

## Loading the dataset

In [4]:
from datasets import load_dataset, load_metric

In [5]:
# If running in Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from datasets import DatasetDict

dataset_split = load_dataset('/content/drive/MyDrive/AI/CLEF2023/custom_dataset', split=[
    'train[0%:80%]', 'validation[80%:100%]'
])

datasets = DatasetDict({'train': dataset_split[0], 'validation': dataset_split[1]})

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset custom_dataset downloaded and prepared to /root/.cache/huggingface/datasets/custom_dataset/customconll2003/1.0.0/e1030d77c373257eb30dc6ce0d3b644340da7c677c804f066114e7e4186d7f0a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 9515
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2379
    })
})

In [8]:
datasets["train"][0]

{'id': '0',
 'tokens': ['Paciente',
  'varón',
  'de',
  'NUMBER',
  'años',
  'de',
  'edad',
  'que',
  'acude',
  'a',
  'la',
  'consulta',
  'de',
  'urología',
  'por',
  'masa',
  'testicular',
  'izquierda'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [10]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PROCEDIMIENTO', 'I-PROCEDIMIENTO']

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [11]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [12]:
show_random_elements(datasets["train"])

,id,tokens,ner_tags
0,8937,"[En, algunos, focos, persistía, una, pequeña, zona, de, bloqueo, central]","[O, O, O, O, O, O, O, O, O, O]"
1,2097,"[No, hay, consanguinidad, y, proceden, de, localidades, diferentes]","[O, O, O, O, O, O, O, O]"
2,1562,"[Realizamos, fulguración, de, las, válvulas, de, la, uretra, posterior, y, orquidectomía, inguinal, derecha]","[O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO]"
3,9254,"[Se, objetivó, también, infestación, por, pediculosis, y, una, lesión, nodular, inflamada, ambas, en, cuero, cabelludo]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4,6399,"[En, la, exploración, fundoscópica, inicial, del, OI, se, aprecia, leve, vitritis, NUMBER, sin, focos, de, retinitis]","[O, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, O, O, O, O, O, O, O, O, O]"
5,2325,"[Durante, el, acto, quirúrgico, se, identificó, una, tumoración, dependiente, del, hueso, con, múltiples, canales, vasculares, dilatados, en, su, seno, que, expandía, la, tabla, externa]","[O, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
6,6693,"[Un, mes, después, es, trasladado, a, nuestro, hospital, con, diagnóstico, de, síndrome, de, lesión, medular, transversa, DNUMBER, ASIA, C, para, evaluación, y, tratamiento, rehabilitador, este, último, aspecto, resultó, imposible, desde, el, primer, momento, por, la, inestabilidad, de, la, lesión, vertebral]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
7,663,"[Procede, de, la, primera, gestación, de, una, madre, sana]","[O, O, O, O, O, O, O, O, O]"
8,7855,"[Nueve, litotricias, cinco, del, lado, derecho, dos, ureterales, y, tres, renales, y, tres, sobre, riñón, izquierdo, aunque, el, hematoma, se, produjo, tras, la, primera, en, este, riñón]","[O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
9,5695,"[El, postoperatorio, transcurrió, sin, complicaciones]","[O, O, O, O, O]"


## Preprocessing the data

In [13]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [14]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [22]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [23]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [24]:
tokenize_and_align_labels(datasets['train'][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[101, 185, 7409, 9080, 1162, 15661, 1320, 1260, 1295, 1126, 2155, 1260, 5048, 3556, 15027, 29005, 10308, 170, 2495, 27231, 1161, 1260, 29009, 12805, 9037, 29087, 12477, 3202, 2774, 20899, 29076, 18276, 25081, 1161, 102], [101, 1185, 5871, 10242, 174, 22650, 22313, 173, 1643, 1964, 102], [101, 8468, 1832, 1186, 1260, 12724, 22904, 10294, 15018, 21713, 11850, 13757, 1389, 6052, 18311, 8468, 12921, 10294, 4060, 9080, 1186, 4315, 102], [101, 175, 4175, 29021, 6575, 2572, 14255, 4298, 29042, 4894, 12416, 1260, 3884, 1179, 8050, 25685, 16468, 16012, 1260, 12563, 1200, 25265, 13166, 1295, 1295, 17713, 4023, 1295, 16358, 7297, 1231, 7136, 16012, 1260, 175, 2155, 29014, 1186, 194, 18463, 1548, 2393, 10401, 1260, 4040, 3556, 23503, 10655, 102], [101, 1185, 171, 13335, 2660, 11437, 1202, 10885, 170, 2495, 3073, 8427, 20295, 1874, 2495, 1805, 28996, 16219, 4490, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [25]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/9515 [00:00<?, ? examples/s]

Map:   0%|          | 0/2379 [00:00<?, ? examples/s]

## Fine-tuning the model

In [26]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at sultan/BioM-BERT-PubMed-PMC-Large were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at sultan/BioM-BERT-PubMed-PMC-Large and are newly initialized: ['classifier.weight', 'cla

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

In [27]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    output_dir = f"/content/drive/MyDrive/AI/CLEF2023/paper_models/80-20-{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy = "epoch",
    push_to_hub=False,
)

In [28]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [29]:
metric = load_metric("seqeval")

<ipython-input-29-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


This metric takes list of labels for the predictions and references:

In [30]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:495: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [31]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Update scheme and mode here
    results = metric.compute(predictions=true_predictions, references=true_labels, scheme="IOB2")
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [32]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [33]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.258000,0.183034,0.782707,0.744924,0.763348,0.940200
2,0.155500,0.173127,0.746356,0.826140,0.784224,0.939026
3,0.104000,0.174728,0.776662,0.821568,0.798484,0.943790


TrainOutput(global_step=1785, training_loss=0.1587481661671016, metrics={'train_runtime': 576.0402, 'train_samples_per_second': 49.554, 'train_steps_per_second': 3.099, 'total_flos': 5900937168880800.0, 'train_loss': 0.1587481661671016, 'epoch': 3.0})

In [34]:
trainer.evaluate()

{'eval_loss': 0.17472775280475616,
 'eval_precision': 0.7766620058472099,
 'eval_recall': 0.8215678364932096,
 'eval_f1': 0.7984840564558285,
 'eval_accuracy': 0.9437904291576824,
 'eval_runtime': 15.7695,
 'eval_samples_per_second': 150.861,
 'eval_steps_per_second': 9.449,
 'epoch': 3.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [35]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'PROCEDIMIENTO': {'precision': 0.7766620058472099,
  'recall': 0.8215678364932096,
  'f1': 0.7984840564558285,
  'number': 7437},
 'overall_precision': 0.7766620058472099,
 'overall_recall': 0.8215678364932096,
 'overall_f1': 0.7984840564558285,
 'overall_accuracy': 0.9437904291576824}

You can now upload the result of the training to the Hub, just execute this instruction: